In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
tf.config.experimental_run_functions_eagerly(True)
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
import math
RANDOM_SEED = 2021
TEST_PCT = 0.3

# Function to split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

alpha_values = list()
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.beta = tf.Variable(0.1)
        self.lstm_units = 14
        self.input_layer = tf.keras.layers.Input(shape=(3, 1))
        self.d0 = tf.keras.layers.LSTM(units=14, activation=self.prelu, return_sequences=True)
        self.d1 = tf.keras.layers.Dropout(0.2)
        self.d2 = tf.keras.layers.LSTM(units=8, activation=self.prelu)
        self.d3 = tf.keras.layers.Dense(1, activation=self.prelu)

    def prelu(self, x):
        ef=x/(1+np.absolute(x))
        return ef * self.beta

    def call(self, x, **kwargs):
        alpha_values.append(self.beta.numpy())
        x = tf.expand_dims(x, axis=-1)  # Add an extra dimension for time_steps
        x = self.d0(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)

        return x

if __name__ == '__main__':
    data = pd.read_csv('yahoo3.csv')
    X_data = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    scaler = StandardScaler()
    X_data = scaler.fit_transform(X_data)

    # Choose the number of time steps
    n_steps = 3

    train_threshold = math.floor(0.7 * len(data))
    X_train, y_train = split_sequence(X_data[:train_threshold][y[:train_threshold] == 0], n_steps)
    X_train = np.squeeze(X_train, axis=-1)
    print(X_train.shape)
    print(y_train.shape)

    X_test, y_test = split_sequence(X_data, n_steps)
    print(X_test.shape)
    X_test = np.squeeze(X_test, axis=-1)
    nb_epoch = 50
    batch_size = 64
    input_dim = X_train.shape[1]  # Adjusted to the number of features after splitting
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.0001,
        patience=10,
        verbose=1,
        mode='min',
        restore_best_weights=True)
    modelq10 = MyModel()
    modelq10.compile(metrics=['accuracy'],loss='mean_squared_error', optimizer='adam')
    X_train = np.array(X_train)
    history=modelq10.fit(X_train, y_train,
                              epochs=nb_epoch,
                              batch_size=batch_size,
                              shuffle=True,
                              verbose=1,
                              callbacks=[ early_stop]
                              ).history
    plt.plot(history['loss'], linewidth=2, label='Train')
    plt.legend(loc='upper right')
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    print(X_test.shape)
    test_x_predictions = modelq10.predict(X_test)
    mse = np.mean(np.power(X_test - test_x_predictions, 2), axis=1)
    print('mse',mse)
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
    ax1.set_title('comparison')
    ax1.set_xlabel('time')
    ax1.set_ylabel('mse')
    ax1.plot(mse, label="dataset")

    ax2.set_xlabel('time')
    ax2.set_ylabel('value')
    ax2.plot(X_test, label="test_data")
    plt.legend(loc='best')
    plt.show()
    mean_mse=np.mean(mse)
    std_mse=np.std(mse)
    lper = np.percentile(mse, 5)
    uper = np.percentile(mse, 95)
    print('lper', lper)
    print('uper', uper)
    y_pred = mse.copy()
    y_pred = np.array(y_pred)
    print(y_pred.shape)
    # Thresholding based on the score
    count = 0
    for itr in range(len(y_pred)):
        if (mse[itr] <= lper or mse[itr] >= uper):
            count = count + 1
            y_pred[itr]=1
        else:
            y_pred[itr]=0

    y=y[3:]
    precision_auto = precision_score(y, y_pred)
    recall_auto = recall_score(y, y_pred)
    f1_score_auto = f1_score(y, y_pred)
    fpr8, tpr8, thresholds = roc_curve(y, y_pred)
    auc_roc_auto = auc(fpr8, tpr8)


    print("Precision: {:.4f}".format(precision_auto))
    print("Recall: {:.4f}".format(recall_auto))
    print("F1-score: {:.4f}".format(f1_score_auto))
    print("AUC-ROC: {:.4f}".format(auc_roc_auto))